# Classification using NLI (Natural language inference)

- 전체 텍스트를 요약하고 요약본의 클래스를 분류한다.
- 원본 글로 분류하면 잘 안됨
- 요약을 하고 난 요약본으로 분류 하면 잘 됨

# 0. Setup

In [1]:
!pip -q install -U transformers langchain

In [1]:
import json
import boto3
import textwrap

from langchain.llms.bedrock import Bedrock
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from transformers import pipeline

In [2]:
profile_name = None
region = 'us-east-1'

In [3]:
session = boto3.Session(
    profile_name=profile_name,
    region_name=region,
)
bedrock = session.client(service_name='bedrock-runtime')

In [4]:
modelId = 'anthropic.claude-v2'

In [5]:
llm = Bedrock(
    model_id=modelId,
    model_kwargs={
        "max_tokens_to_sample": 4096,
        "top_p": 0.9,
        "temperature": 0,
    },
    client=bedrock,
)

# 1. Data Load

- [Anti-corruption layer pattern](https://docs.aws.amazon.com/prescriptive-guidance/latest/cloud-design-patterns/acl.html)

In [6]:
with open('acl.txt', 'r') as fp:
    doc = fp.read()
textwrap.shorten(doc, width=80, placeholder=' ...')

'Anti-corruption layer pattern Intent The anti-corruption layer (ACL) pattern ...'

# 2. Summarize

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n"],
    chunk_size=3000,
    chunk_overlap=100,
)
docs = text_splitter.create_documents([doc])

In [8]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    verbose=False,
)

In [9]:
output = summary_chain(docs)

In [10]:
summary = output['output_text'].strip()
print(summary)

Here is a concise summary of the key points:

The anti-corruption layer (ACL) pattern provides an abstraction between a monolithic application and a microservice. It translates calls between the incompatible interfaces, allowing incremental migration without disrupting the monolith. The ACL can be implemented as a facade within the monolith or an independent service. It enables decoupling but adds overhead. The ACL should be decommissioned after full migration. Key considerations are technical debt, latency, scaling, and implementation strategy. The provided C# code demonstrates an ACL implementation that transforms monolith data models into the microservice format and handles integration. The ACL isolates the monolith from microservice changes during migration.


# 3. Classify using NLI

- nli 모델로 텍스트 카테고리 분류

In [24]:
classifier = pipeline(
    task='zero-shot-classification',
    model='facebook/bart-large-mnli',
)

In [25]:
candidate_labels = ['software engineer', 'web designer', 'digital marketer']

In [26]:
len(doc)

9966

In [27]:
%%time

res = classifier(doc, candidate_labels)
print(f'{json.dumps(res, indent=2)}\n')

{
  "sequence": "Anti-corruption layer pattern\n\nIntent\nThe anti-corruption layer (ACL) pattern acts as a mediation layer that translates domain model semantics from one system to another system. It translates the model of the upstream bounded context (monolith) into a model that suits the downstream bounded context (microservice) before consuming the communication contract that's established by the upstream team. This pattern might be applicable when the downstream bounded context contains a core subdomain, or the upstream model is an unmodifiable legacy system. It also reduces transformation risk and business disruption by preventing changes to callers when their calls have to be redirected transparently to the target system.\nMotivation\n\nDuring the migration process, when a monolithic application is migrated into microservices, there might be changes in the domain model semantics of the newly migrated service. When the features within the monolith are required to call these micr

In [28]:
len(summary)

772

In [29]:
%%time

res = classifier(summary, candidate_labels)
print(f'{json.dumps(res, indent=2)}\n')

{
  "sequence": "Here is a concise summary of the key points:\n\nThe anti-corruption layer (ACL) pattern provides an abstraction between a monolithic application and a microservice. It translates calls between the incompatible interfaces, allowing incremental migration without disrupting the monolith. The ACL can be implemented as a facade within the monolith or an independent service. It enables decoupling but adds overhead. The ACL should be decommissioned after full migration. Key considerations are technical debt, latency, scaling, and implementation strategy. The provided C# code demonstrates an ACL implementation that transforms monolith data models into the microservice format and handles integration. The ACL isolates the monolith from microservice changes during migration.",
  "labels": [
    "software engineer",
    "web designer",
    "digital marketer"
  ],
  "scores": [
    0.692902147769928,
    0.1681481897830963,
    0.13894963264465332
  ]
}

CPU times: user 1.59 s, sys

# 4. Korean

- 한글지원 nli 모델은 [여기](https://huggingface.co/models?pipeline_tag=zero-shot-classification&language=ko&sort=trending)서 찾을 수 있다.
- 예제에서는 [MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7](https://huggingface.co/MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7) 모델을 사용한다.

In [49]:
ko_classifier = pipeline(
    task='zero-shot-classification',
    model='MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7',
)

- 배송과 포장에 대해서는 약한 불만이 있지만 제품 자체에 대해서는 큰 만족도를 가진 리뷰

In [54]:
sequence = '''
새벽 배송이라고 써있어서 새벽에 올 줄 알았는데, 오후 늦게 도착해서 실망했습니다.
아이폰 박스에 뽁뽁이로 포장이 안되어 있어서 제품 파손이 약간 우려스러웠습니다.
제가 아이폰만 계속 쓰고 있어서 그런것도 있지만, 제품은 정말 최고의 제품입니다.
아직 안바꾸신 분들 계시면 꼭 바꾸시길 추천합니다.
'''
candidate_labels =['긍정', '부정', '중립']

- 완전한 제로샷 분류의 경우 전체적으로는 부정이라고 판단하고 있다. 섞여있기 때문에 강하게 확신하지 못한다.
- 제품에 대한 만족감을 템플릿으로 설정하면 긍정이라고 크게 확신한다.
- 배송에 대한 만족감을 템플릿으로 설정하면 부정이라고 크게 확신한다.

In [55]:
ko_classifier(
    sequence,
    candidate_labels,
)

{'sequence': '\n새벽 배송이라고 써있어서 새벽에 올 줄 알았는데, 오후 늦게 도착해서 실망했습니다.\n아이폰 박스에 뽁뽁이로 포장이 안되어 있어서 제품 파손이 약간 우려스러웠습니다.\n제가 아이폰만 계속 쓰고 있어서 그런것도 있지만, 제품은 정말 최고의 제품입니다.\n아직 안바꾸신 분들 계시면 꼭 바꾸시길 추천합니다.\n',
 'labels': ['부정', '긍정', '중립'],
 'scores': [0.5180058479309082, 0.2957472503185272, 0.1862468123435974]}

In [56]:
ko_classifier(
    sequence,
    candidate_labels,
    hypothesis_template='제품에 대한 만족감은 {} 이다.',
)

{'sequence': '\n새벽 배송이라고 써있어서 새벽에 올 줄 알았는데, 오후 늦게 도착해서 실망했습니다.\n아이폰 박스에 뽁뽁이로 포장이 안되어 있어서 제품 파손이 약간 우려스러웠습니다.\n제가 아이폰만 계속 쓰고 있어서 그런것도 있지만, 제품은 정말 최고의 제품입니다.\n아직 안바꾸신 분들 계시면 꼭 바꾸시길 추천합니다.\n',
 'labels': ['긍정', '부정', '중립'],
 'scores': [0.7716712951660156, 0.1566908359527588, 0.07163792103528976]}

In [57]:
ko_classifier(
    sequence,
    candidate_labels,
    hypothesis_template='배송에 대한 만족감은 {} 이다.',
)

{'sequence': '\n새벽 배송이라고 써있어서 새벽에 올 줄 알았는데, 오후 늦게 도착해서 실망했습니다.\n아이폰 박스에 뽁뽁이로 포장이 안되어 있어서 제품 파손이 약간 우려스러웠습니다.\n제가 아이폰만 계속 쓰고 있어서 그런것도 있지만, 제품은 정말 최고의 제품입니다.\n아직 안바꾸신 분들 계시면 꼭 바꾸시길 추천합니다.\n',
 'labels': ['부정', '긍정', '중립'],
 'scores': [0.8069165349006653, 0.11074954271316528, 0.08233391493558884]}